In [1]:
import polars as pl
from pathlib import Path
import matplotlib.pyplot as plt

In [9]:
# Set the column width to 1000
pl.Config.set_fmt_str_lengths(1000)

polars.config.Config

In [3]:
base = Path("/Users/larsankile/code/defi-measurement/data")

In [4]:
pools = pl.read_parquet(base / "factory_pool_created/*.parquet")

pools.columns

['chain_name',
 'block_timestamp',
 'block_number',
 'transaction_hash',
 'log_index',
 'token0',
 'token1',
 'fee',
 'tickSpacing',
 'pool']

In [5]:
tokens = pl.read_parquet(base / "uniswap-v3-tokens.parquet")

tokens.columns

['address', 'symbol', 'name', 'decimals']

In [12]:
# Prepare token_info_df by selecting only the columns we need and transforming address to lowercase
token_info = tokens.select(pl.col("address").str.to_lowercase().alias("address"), pl.col("symbol"), pl.col("decimals"))

# Join for token0 information, transforming token0 to lowercase
result = pools.join(token_info, left_on=pl.col("token0").str.to_lowercase(), right_on="address", how="left").rename(
    {"symbol": "symbol0", "decimals": "decimals0"}
)

# Join for token1 information, transforming token1 to lowercase
result = result.join(token_info, left_on=pl.col("token1").str.to_lowercase(), right_on="address", how="left").rename(
    {"symbol": "symbol1", "decimals": "decimals1"}
)

# Exclude the address columns
result = result.select(pl.all().exclude(columns=["address", "address_right"]))

result

chain_name,block_timestamp,block_number,transaction_hash,log_index,token0,token1,fee,tickSpacing,pool,symbol0,decimals0,symbol1,decimals1
str,"datetime[μs, UTC]",i64,str,i64,str,str,f64,str,str,str,f64,str,f64
"""ethereum""",2023-05-25 13:12:59 UTC,17336379,"""0x4e47ed4be1863c6154d58933f58f9ee066c667e56704c8094c67e1828234c3fb""",0,"""0x28c6ce090bf0d534815c59440a197e92b4cf718f""","""0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2""",10000.0,"""200""","""0x2476e13c3da1e0daf898a0b0d204226f2b857c2b""",null,null,"""WETH""",18.0
"""ethereum""",2023-05-25 22:14:35 UTC,17339057,"""0x9e6db41b51720670c67021bf699446c8d4e85e6ce3f25da4b8d240e6a5152f47""",417,"""0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2""","""0xd741af14d4ef05a094c812611e848dbfef90981c""",3000.0,"""60""","""0x448a70a160d6be76ec6cdf3a7ed9988d3eeebcd2""","""WETH""",18.0,null,null
"""ethereum""",2023-05-25 15:39:11 UTC,17337101,"""0xce1132de4d03a15a9b98a79d9543cb06c1017b2fb88e7f98ad8b18f94a232273""",350,"""0x8ee08493373c48cf6923efd0d0573ff504f733b0""","""0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2""",100.0,"""1""","""0x33b8fe34bf752ea0971fcba65ac994f2c04ed6e4""",null,null,"""WETH""",18.0
"""ethereum""",2023-05-25 19:57:35 UTC,17338379,"""0x37310c5f5a2442798a8ac6ea98034885da32fe15960f47b622aab38a5c80bac1""",0,"""0x10ca7856ec495c112c783d17626c597cf7ffd977""","""0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2""",10000.0,"""200""","""0xf4eeb4ad7b13b981e95cbd571eca9e476b3d5412""",null,null,"""WETH""",18.0
"""ethereum""",2023-05-25 05:55:23 UTC,17334222,"""0x9c719325a9a1a6893d710d0267c67067ab27a56b72a9d686f4a0cc56b524c839""",453,"""0xb7ea5d7528a9f86368ff49524ed293f4266d11f2""","""0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2""",10000.0,"""200""","""0x262d0733b5e1440b1ce514b46dfc0184ca6a8eab""",null,null,"""WETH""",18.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""ethereum""",2024-07-06 17:12:59 UTC,20248886,"""0x6a17c8e21d95b487e772243bbefaf671378b2f62742de9ef830e47acef3209aa""",7,"""0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2""","""0xf0d76555b129f88eadce08367bad18af7abdf47f""",10000.0,"""200""","""0x4a5d841ca2de24073d224c613310cd7266be5a02""","""WETH""",18.0,null,null
"""ethereum""",2024-07-06 17:28:35 UTC,20248964,"""0x8b8ec6ac0a4cd625ea8ef68db7410df997dc2f24783ec95adbdcf6fd0078868b""",43,"""0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2""","""0xc62b229862e2b1689f281d44cbb95fb416ab69b6""",100.0,"""1""","""0xd1054cdb9ec83e7929eccd1ca3d2f765dabe2a56""","""WETH""",18.0,null,null
"""ethereum""",2024-07-06 18:55:47 UTC,20249399,"""0x412977292aea40df6769d191e3db517b46af7f2da770346118abea5bab8e5096""",42,"""0x0403a52fcb05e303bfc0246c5861a81ab09c1e8e""","""0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2""",10000.0,"""200""","""0x51947a286c484962372952bdf1d0fac2ec13b9d4""",null,null,"""WETH""",18.0


In [13]:
filtered_result = result.filter(pl.col("decimals0").is_not_null() & pl.col("decimals1").is_not_null())

filtered_result

chain_name,block_timestamp,block_number,transaction_hash,log_index,token0,token1,fee,tickSpacing,pool,symbol0,decimals0,symbol1,decimals1
str,"datetime[μs, UTC]",i64,str,i64,str,str,f64,str,str,str,f64,str,f64
"""ethereum""",2023-05-25 23:05:47 UTC,17339307,"""0x2f31841a5174d4470bd375358327473d735ec0be94718924fd39643f81b5c62a""",198,"""0x6cf4cf8f827b350c474c3862dbf13298977ee42d""","""0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2""",10000.0,"""200""","""0x7bf4eaced115b261e02737bd656571e4bc335549""","""REV""",18.0,"""WETH""",18.0
"""ethereum""",2023-05-25 22:59:59 UTC,17339278,"""0x1334b6ea3d67bf7d0913764eb739b22fb4d7231f790b635ed7a6acce2dbc61ab""",299,"""0x40063963486c17be51743a18c016bfbd07e7ca15""","""0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2""",3000.0,"""60""","""0x5081c559e31403b7e77532a5c3a2099d3da5e672""","""RBE""",18.0,"""WETH""",18.0
"""ethereum""",2023-05-25 09:01:47 UTC,17335142,"""0x448fb8123f1d14109d2cd042332ab6e7596315fb02f07c14e39f82157b7d07bc""",264,"""0x0a4024d7ff45df2f9488649af70491d6beee6e80""","""0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2""",3000.0,"""60""","""0x28f354bbb13f7d6738424cd3c092607c8f907999""","""MRW""",18.0,"""WETH""",18.0
"""ethereum""",2023-05-25 15:45:59 UTC,17337135,"""0x39cda277528951896de1307c245991b5abb9fdb4672af7b07700d5aa06f3e1f0""",222,"""0x320dc0b6aafeb9fa7106799c9bbea3f02dde1f56""","""0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2""",10000.0,"""200""","""0x454743fb44e3d9fb1e21cc4b74e462e601140a49""","""CENSOR""",18.0,"""WETH""",18.0
"""ethereum""",2023-05-25 07:43:11 UTC,17334755,"""0xba1b2b6941763e2c2d32065fec3a50a42e2628a2c138215354987786995dfad9""",173,"""0x3cbbd2511139fd37497a5d01321b6aaf52c2b42f""","""0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2""",10000.0,"""200""","""0x2186bb4bad31902290a64e30912c3d8360a7ecbc""","""SQUID""",18.0,"""WETH""",18.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""ethereum""",2024-07-01 23:09:23 UTC,20214859,"""0xaae81e0efcfec01262024e2e7b070c1183ccba27672b1a5970d7804f42b7a23c""",0,"""0x0abdace70d3790235af448c88547603b945604ea""","""0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2""",10000.0,"""200""","""0x74f3710792b88bce2c80698efc8a7829ad37d777""","""DNT""",18.0,"""WETH""",18.0
"""ethereum""",2024-07-02 17:28:35 UTC,20220320,"""0x2995d0178413cb29f91fa464aae4d7e8f4aa5524de9e5a932ef92a4adb8afe44""",394,"""0x6ca9d0b650c7a4b3871ea898b9f1a7fd27fb1d58""","""0xdac17f958d2ee523a2206206994597c13d831ec7""",100.0,"""1""","""0xe1ab3ac0495f991d2295b760418e255f4343571a""","""USD""",18.0,"""USDT""",6.0
"""ethereum""",2024-07-02 02:29:47 UTC,20215859,"""0xfa0326f6103cd8eb15e375a86f9afec5385728a3592f3424a52051f38fdaef66""",522,"""0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2""","""0xf230b790e05390fc8295f4d3f60332c93bed42e2""",10000.0,"""200""","""0x6792ff48f3164030f09f21cc5894fd5644418c30""","""WETH""",18.0,"""TRX""",6.0


In [14]:
# Save the result to a parquet file
filtered_result.write_parquet(base / "pool_token_info.parquet")